The goal for these simulation is to illustrate role of polarity transmission in the migratory mode emergence. 

Polarity local transmission is defined independently. Beta = 0,0.3,0.6,0.9 is simulated accordingly. 

This jupyter notebook submit the simulation job to the cluster.

In [1]:
import numpy as np
import sys
from datetime import datetime
import subprocess
import os

import pipe_util2

In [2]:
def parameter_txt(rep, param_folder, scan_num, 
                  input_path, output_path
                 ):
    """
    For each scan repeat, generate a folder and parameter files
    For parallel processing on cluster
    
    rep - int, # of repeats
    param_folder - string
    scan_num - int, # of simulations per scan
    input_path - string, path to model force_fpp_gl.cc3d file
    output_path - string, path to save simulated data
    """
    param_folder = pipe_util2.folder_verify(param_folder)
    
    i = 0 #total simulation count
    while i<rep*scan_num:
        
        j = 0 #repeat count
        while j<rep:
            output_path = pipe_util2.folder_verify(output_path)
            o_path = output_path+"rep%s/"%str(j).zfill(2)
            pipe_util2.create_folder(o_path)
            
            k = 0 #scan count
            while k<scan_num:
                
                file_num = str(i)
                f = open(param_folder+file_num+".p", "w")
            
                line = "/net/capricorn/home/xing/huijing/.conda/envs/cc3d_460_310/bin/python -m cc3d.core.param_scan.parameter_scan_run --input=%s --output-dir=%s --install-dir=''"%(
                    input_path, o_path
                )

                f.write(line)
                f.close()
                
                i = i+1
                k = k+1
            j = j+1
    return i-1

In [3]:
def sbatch_file_polarity_sim(
    param_folder, scr_folder, name = None):
    """
    Generate the .sh file for submitting job array to the cluster
    
    param_folder - string, path to folder that keep parameters
    scr_folder - string, path to folder that keep .sh and .o files
    """
    
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d-%H%M")
    scr_folder = pipe_util2.folder_verify(scr_folder)
    param_folder = pipe_util2.folder_verify(param_folder)
    
    if name:
        ofile_name = scr_folder + dt_string + "polarity_sim_" + name + ".o"
        sfile_name = scr_folder + dt_string + "polarity_sim_" + name + ".sh"
        
    else:
        ofile_name = scr_folder + dt_string + "polarity_sim" + ".o"
        sfile_name = scr_folder + dt_string + "polarity_sim" + ".sh"
    
    ofile_name = os.path.abspath(ofile_name)
    sfile_name = os.path.abspath(sfile_name)
    
    contents = """#!/bin/bash
#SBATCH --account=huijing
#SBATCH --partition=dept_cpu
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH --output=%s%s_%s.o
#SBATCH --job-name=potts_scan


bash %s$SLURM_ARRAY_TASK_ID.p
    """%(scr_folder, "%A", "%a", param_folder)
    s = open(sfile_name, "w")
    s.write(contents)
    s.close()
    
    return sfile_name

In [4]:
sim_name_list = [
    "20250129_b0_3_force_fpp_scan",
    "20250129_b0_6_force_fpp_scan",
    "20250129_b0_force_fpp_scan",
    "20250129_b1_force_fpp_scan"
]

In [5]:
rep = 50
# total # of parameter scan if repeat once for each
# param 1 # x param 2 #
scan_num = 231 


# generating comand line for running simulation on cluster
cmd_list = []

for sim_name in sim_name_list:
    
    # define number of jobs that run at the same time 
    batch_size = 300

    param_folder = "%s_param"%sim_name
    pipe_util2.create_folder(param_folder)

    script_folder = "%s_script"%sim_name
    pipe_util2.create_folder(script_folder)

    out_dir = "/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_%s/"%sim_name
    pipe_util2.create_folder(out_dir)

    input_path = "/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model/%s/force_fpp_gl.cc3d"%sim_name

    jobmax_num = parameter_txt(rep, param_folder, scan_num, 
                               input_path, out_dir)
    print("total # of simulations: %s"%jobmax_num)
    sbatch_file = sbatch_file_polarity_sim(os.path.abspath(param_folder), os.path.abspath(script_folder))

    # if the number of simulation is less than the batch size
    # use a smaller batch size.
    if jobmax_num<batch_size:
        batch_size = jobmax_num+1
    batch_size = "%"+"%s"%batch_size
    cmd = "sbatch --array=[0-%s]%s %s"%(jobmax_num, batch_size, sbatch_file)
    #subprocess.call(cmd, shell=True)
    cmd_list.append(cmd)
    print(cmd)

20250129_b0_3_force_fpp_scan_param/ folder is freshly created. 

20250129_b0_3_force_fpp_scan_script/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep00/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep01/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep02/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep03/ folder is freshly c

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep45/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep46/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep47/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep48/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_3_force_fpp_scan/rep49/ folder is freshly created. 

total # of simulations: 11549
sbatch --array=[0-11549]%300 /net/capricorn/home/xing/huijing/Migration/20250128_Supp

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_6_force_fpp_scan/rep40/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_6_force_fpp_scan/rep41/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_6_force_fpp_scan/rep42/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_6_force_fpp_scan/rep43/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_6_force_fpp_scan/rep44/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_202

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_force_fpp_scan/rep34/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_force_fpp_scan/rep35/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_force_fpp_scan/rep36/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_force_fpp_scan/rep37/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_force_fpp_scan/rep38/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b0_f

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b1_force_fpp_scan/rep30/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b1_force_fpp_scan/rep31/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b1_force_fpp_scan/rep32/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b1_force_fpp_scan/rep33/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b1_force_fpp_scan/rep34/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-sim-data/CC3D_data_20250129_b1_f

In [6]:
cmd_list

['sbatch --array=[0-11549]%300 /net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129_b0_3_force_fpp_scan_script/2025-01-30-1225polarity_sim.sh',
 'sbatch --array=[0-11549]%300 /net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129_b0_6_force_fpp_scan_script/2025-01-30-1225polarity_sim.sh',
 'sbatch --array=[0-11549]%300 /net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129_b0_force_fpp_scan_script/2025-01-30-1225polarity_sim.sh',
 'sbatch --array=[0-11549]%300 /net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129_b1_force_fpp_scan_script/2025-01-30-1225polarity_sim.sh']

In [7]:
# submitting job array for simulations

for i in range(len(cmd_list)):
    subprocess.call(cmd_list[i], shell=True)

Submitted batch job 44828486
Submitted batch job 44828487
Submitted batch job 44828488
Submitted batch job 44828489
